In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
# Here I changed the no of filters as by compressing the no of filters will be decreased again, so we took more no of filters.
# No of layers we changed the to half
num_classes = 10
epochs = 50
l = 20
num_filter = 64
compression = 0.5
dropout_rate = 0.3

In [6]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

145022976/170498071 [========================>.....] - ETA: 7s

170500096/170498071 [==============================] - 54s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 64, dropout_rate = 0.3):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 64, dropout_rate = 0.3):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 64
dropout_rate = 0.3
l = 20
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [11]:
# Checkpoint is simply used for the model to save the weights if anf only if the validation accuracy is improved.
# Reduce_lr -> This callback is used for if there is no improvment seen for patience then we reduce the learning rate.

from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
model = Model(inputs=[input], outputs=[output])
checkpoint=ModelCheckpoint("session4b.hdf5", monitor='val_acc', verbose=1, save_best_only=True, mode='max', period=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2,verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.001)
callbacks_list=[checkpoint, reduce_lr]
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   1728        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [13]:
# Her I took the epochs in 5 intervals, so that I can understand how much the accuracy for interval and also easy to save the weights.
epoch = 0
epochs = 5

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    initial_epoch=epoch,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

model.save_weights("DNST_model.h5")
print("Saved model to disk")

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 495s 10ms/step - loss: 1.4454 - acc: 0.4681 - val_loss: 1.4745 - val_acc: 0.5066

Epoch 00001: val_acc improved from -inf to 0.50660, saving model to session4b.hdf5
Epoch 2/5
14848/50000 [=======>......................] - ETA: 5:07 - loss: 1.0625 - acc: 0.6199

50000/50000 [==============================] - 473s 9ms/step - loss: 0.9719 - acc: 0.6527 - val_loss: 1.2929 - val_acc: 0.6243

Epoch 00002: val_acc improved from 0.50660 to 0.62430, saving model to session4b.hdf5
Epoch 3/5
31488/50000 [=================>............] - ETA: 2:41 - loss: 0.8109 - acc: 0.7116

50000/50000 [==============================] - 471s 9ms/step - loss: 0.7920 - acc: 0.7187 - val_loss: 1.5926 - val_acc: 0.6043

Epoch 00003: val_acc did not improve from 0.62430
Epoch 4/5
42752/50000 [========================>.....] - ETA: 1:02 - loss: 0.6926 - acc: 0.7568

50000/50000 [==============================] - 470s 9ms/step - loss: 0.6886 - acc: 0.7586 - val_loss: 1.5738 - val_acc: 0.6271

Epoch 00004: val_acc improved from 0.62430 to 0.62710, saving model to session4b.hdf5
Epoch 5/5
39168/50000 [======================>.......] - ETA: 1:34 - loss: 0.6124 - acc: 0.7841

50000/50000 [==============================] - 470s 9ms/step - loss: 0.6132 - acc: 0.7845 - val_loss: 0.9002 - val_acc: 0.7380

Epoch 00005: val_acc improved from 0.62710 to 0.73800, saving model to session4b.hdf5
Saved model to disk


In [15]:
epoch = 5
epochs = 10

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    initial_epoch=epoch,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

model.save_weights("DNST_model.h5")
print("Saved model to disk")

Train on 50000 samples, validate on 10000 samples
Epoch 6/10
50000/50000 [==============================] - 470s 9ms/step - loss: 0.5024 - acc: 0.8255 - val_loss: 1.5512 - val_acc: 0.6467

Epoch 00006: val_acc did not improve from 0.73800
Epoch 7/10
17280/50000 [=========>....................] - ETA: 4:44 - loss: 0.4567 - acc: 0.8413

50000/50000 [==============================] - 470s 9ms/step - loss: 0.4594 - acc: 0.8400 - val_loss: 0.6155 - val_acc: 0.8183

Epoch 00007: val_acc improved from 0.73800 to 0.81830, saving model to session4b.hdf5
Epoch 8/10
32000/50000 [==================>...........] - ETA: 2:36 - loss: 0.4319 - acc: 0.8497

50000/50000 [==============================] - 470s 9ms/step - loss: 0.4347 - acc: 0.8485 - val_loss: 1.0611 - val_acc: 0.7365

Epoch 00008: val_acc did not improve from 0.81830
Epoch 9/10
40960/50000 [=======================>......] - ETA: 1:18 - loss: 0.4048 - acc: 0.8582

50000/50000 [==============================] - 470s 9ms/step - loss: 0.4074 - acc: 0.8575 - val_loss: 0.7519 - val_acc: 0.7766

Epoch 00009: val_acc did not improve from 0.81830
Epoch 10/10
44544/50000 [=========================>....] - ETA: 47s - loss: 0.3790 - acc: 0.8677

50000/50000 [==============================] - 470s 9ms/step - loss: 0.3784 - acc: 0.8682 - val_loss: 0.8494 - val_acc: 0.7873

Epoch 00010: val_acc did not improve from 0.81830
Saved model to disk


In [18]:
epoch = 10
epochs = 15

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    initial_epoch=epoch,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))


model.save_weights("DNST_model.h5")
print("Saved model to disk")

Train on 50000 samples, validate on 10000 samples
Epoch 11/15
50000/50000 [==============================] - 472s 9ms/step - loss: 0.2252 - acc: 0.9198 - val_loss: 0.7850 - val_acc: 0.8172

Epoch 00011: val_acc did not improve from 0.82540
Epoch 12/15
16000/50000 [========>.....................] - ETA: 4:57 - loss: 0.2028 - acc: 0.9277

50000/50000 [==============================] - 473s 9ms/step - loss: 0.2095 - acc: 0.9248 - val_loss: 0.8026 - val_acc: 0.8208

Epoch 00012: val_acc did not improve from 0.82540
Epoch 13/15
35840/50000 [====================>.........] - ETA: 2:03 - loss: 0.1985 - acc: 0.9292

50000/50000 [==============================] - 471s 9ms/step - loss: 0.2035 - acc: 0.9276 - val_loss: 0.6616 - val_acc: 0.8481

Epoch 00013: val_acc improved from 0.82540 to 0.84810, saving model to session4b.hdf5
Epoch 14/15
36992/50000 [=====================>........] - ETA: 1:53 - loss: 0.1886 - acc: 0.9331

50000/50000 [==============================] - 472s 9ms/step - loss: 0.1900 - acc: 0.9321 - val_loss: 0.8569 - val_acc: 0.8184

Epoch 00014: val_acc did not improve from 0.84810
Epoch 15/15
42368/50000 [========================>.....] - ETA: 1:06 - loss: 0.1883 - acc: 0.9337

50000/50000 [==============================] - 473s 9ms/step - loss: 0.1892 - acc: 0.9335 - val_loss: 0.7166 - val_acc: 0.8369

Epoch 00015: val_acc did not improve from 0.84810
Saved model to disk


In [21]:
# Here I changed the intervals of epoch to 10 intervals as there is no thst much of improve in the valiation accuracy.
epoch = 15
epochs = 25

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    initial_epoch=epoch,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

model.save_weights("DNST_model.h5")
print("Saved model to disk")

Train on 50000 samples, validate on 10000 samples
Epoch 16/25
50000/50000 [==============================] - 470s 9ms/step - loss: 0.0953 - acc: 0.9660 - val_loss: 0.6479 - val_acc: 0.8716

Epoch 00016: val_acc improved from 0.86860 to 0.87160, saving model to session4b.hdf5
Epoch 17/25
14336/50000 [=======>......................] - ETA: 5:09 - loss: 0.0777 - acc: 0.9724

50000/50000 [==============================] - 470s 9ms/step - loss: 0.0911 - acc: 0.9671 - val_loss: 0.8326 - val_acc: 0.8469

Epoch 00017: val_acc did not improve from 0.87160
Epoch 18/25
35328/50000 [====================>.........] - ETA: 2:06 - loss: 0.0894 - acc: 0.9689

50000/50000 [==============================] - 468s 9ms/step - loss: 0.0937 - acc: 0.9668 - val_loss: 0.5989 - val_acc: 0.8800

Epoch 00018: val_acc improved from 0.87160 to 0.88000, saving model to session4b.hdf5
Epoch 19/25
36864/50000 [=====================>........] - ETA: 1:53 - loss: 0.0883 - acc: 0.9684

50000/50000 [==============================] - 467s 9ms/step - loss: 0.0921 - acc: 0.9668 - val_loss: 0.9540 - val_acc: 0.8345

Epoch 00019: val_acc did not improve from 0.88000
Epoch 20/25
42240/50000 [========================>.....] - ETA: 1:06 - loss: 0.0870 - acc: 0.9687

50000/50000 [==============================] - 467s 9ms/step - loss: 0.0881 - acc: 0.9682 - val_loss: 0.6703 - val_acc: 0.8696

Epoch 00020: val_acc did not improve from 0.88000
Epoch 21/25
43904/50000 [=========================>....] - ETA: 52s - loss: 0.0817 - acc: 0.9712

50000/50000 [==============================] - 468s 9ms/step - loss: 0.0829 - acc: 0.9711 - val_loss: 0.6198 - val_acc: 0.8794

Epoch 00021: val_acc did not improve from 0.88000
Epoch 22/25
44416/50000 [=========================>....] - ETA: 48s - loss: 0.0831 - acc: 0.9695

50000/50000 [==============================] - 467s 9ms/step - loss: 0.0837 - acc: 0.9695 - val_loss: 0.8899 - val_acc: 0.8398

Epoch 00022: val_acc did not improve from 0.88000
Epoch 23/25
44672/50000 [=========================>....] - ETA: 46s - loss: 0.0815 - acc: 0.9706

50000/50000 [==============================] - 469s 9ms/step - loss: 0.0816 - acc: 0.9708 - val_loss: 0.6916 - val_acc: 0.8721

Epoch 00023: val_acc did not improve from 0.88000
Epoch 24/25
44672/50000 [=========================>....] - ETA: 46s - loss: 0.0759 - acc: 0.9729

50000/50000 [==============================] - 468s 9ms/step - loss: 0.0766 - acc: 0.9726 - val_loss: 0.6842 - val_acc: 0.8696

Epoch 00024: val_acc did not improve from 0.88000
Epoch 25/25
44672/50000 [=========================>....] - ETA: 46s - loss: 0.0716 - acc: 0.9744

50000/50000 [==============================] - 468s 9ms/step - loss: 0.0734 - acc: 0.9740 - val_loss: 0.6696 - val_acc: 0.8705

Epoch 00025: val_acc did not improve from 0.88000
Saved model to disk


In [0]:
epoch = 25
epochs = 35

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    initial_epoch=epoch,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

model.save_weights("DNST_model.h5")
print("Saved model to disk")

Train on 50000 samples, validate on 10000 samples
Epoch 26/35
50000/50000 [==============================] - 468s 9ms/step - loss: 0.0792 - acc: 0.9715 - val_loss: 0.7675 - val_acc: 0.8583

Epoch 00026: val_acc did not improve from 0.88000
Epoch 27/35
16000/50000 [========>.....................] - ETA: 4:54 - loss: 0.0725 - acc: 0.9744

50000/50000 [==============================] - 468s 9ms/step - loss: 0.0720 - acc: 0.9746 - val_loss: 0.6649 - val_acc: 0.8738

Epoch 00027: val_acc did not improve from 0.88000
Epoch 28/35
35840/50000 [====================>.........] - ETA: 2:02 - loss: 0.0700 - acc: 0.9758

50000/50000 [==============================] - 468s 9ms/step - loss: 0.0708 - acc: 0.9751 - val_loss: 0.7961 - val_acc: 0.8551

Epoch 00028: val_acc did not improve from 0.88000
Epoch 29/35
41984/50000 [========================>.....] - ETA: 1:09 - loss: 0.0697 - acc: 0.9751

50000/50000 [==============================] - 468s 9ms/step - loss: 0.0700 - acc: 0.9752 - val_loss: 0.7623 - val_acc: 0.8552

Epoch 00029: val_acc did not improve from 0.88000
Epoch 30/35
43904/50000 [=========================>....] - ETA: 52s - loss: 0.0695 - acc: 0.9751

50000/50000 [==============================] - 468s 9ms/step - loss: 0.0688 - acc: 0.9753 - val_loss: 0.6317 - val_acc: 0.8810

Epoch 00030: val_acc improved from 0.88000 to 0.88100, saving model to session4b.hdf5
Epoch 31/35
39168/50000 [======================>.......] - ETA: 1:33 - loss: 0.0638 - acc: 0.9774

50000/50000 [==============================] - 468s 9ms/step - loss: 0.0643 - acc: 0.9772 - val_loss: 0.7407 - val_acc: 0.8746

Epoch 00031: val_acc did not improve from 0.88100
Epoch 32/35
43008/50000 [========================>.....] - ETA: 1:00 - loss: 0.0700 - acc: 0.9757

50000/50000 [==============================] - 468s 9ms/step - loss: 0.0700 - acc: 0.9754 - val_loss: 0.8077 - val_acc: 0.8675

Epoch 00032: val_acc did not improve from 0.88100
Epoch 33/35
28544/50000 [================>.............] - ETA: 3:05 - loss: 0.0665 - acc: 0.9770

In [0]:
# The network got disconnected and unable to run the epochs as the colab was crashing and I got the validation accuracy is 88.1.
# As there is no improvemetn during the epochs from 18 to 31 and it increased to 0.1 percent and I stopped running 33rd epoch.